# Modeling & Evaluation

Inhaltsverzeichnis

In [1]:
%load_ext autoreload
%autoreload 2

---

## Topic Modeling

### Modeling

In [2]:
import pandas as pd

# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')
df.head(4)

,url,date,rawContent,preprocessed_text
0,https://twitter.com/YueDongCS/status/164159107...,2023-03-30 23:59:46,My condolences and sad that the #NLP and #AI c...,"[condolence, sad, nlp, ai, community, lose, br..."
1,https://twitter.com/Mlearning_ai/status/164159...,2023-03-30 23:59:43,Hiring Now: The Top Jobs of the Future Fueled ...,"[hiring, top, job, future, fueled, generative,..."
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:43,What’s the hack. #stem #science #stemeducation...,"[hack, stem, science, stemeducation, education..."
3,https://twitter.com/Stemble_/status/1641590942...,2023-03-30 23:59:14,"🚀 Mark your calendars, Apple enthusiasts! 🗓️\n...","[mark, calendar, apple, enthusiast, june, 5, a..."


#### 1. Erstellen eines Wörterbuchs

Ein Wörterbuch ist wichtig für das LDA-Modell, da es das Modell trainiert, die Sprache und den Kontext der Dokumente zu verstehen. Es enthält alle eindeutigen Wörter, die in den Dokumenten vorkommen, und ordnet jedem Wort eine eindeutige Nummer zu. Das Wörterbuch wird verwendet, um jedes Dokument in eine Vektordarstellung umzuwandeln, die das Modell verwenden kann.

In [3]:
from gensim import corpora

dictionary = corpora.Dictionary(df['preprocessed_text'])

#### 2. Erstellen eines Corpus

Ein Corpus ist notwendig, um jedes Dokument in eine Vektordarstellung umzuwandeln, die vom LDA-Modell verarbeitet werden kann. Ein Corpus ist eine Sammlung von Dokumenten, die in eine Matrix umgewandelt wird, wobei jede Zeile für ein Dokument und jede Spalte für ein Wort im Wörterbuch steht. Die Matrix enthält die Anzahl der Vorkommen jedes Wortes in jedem Dokument. Durch die Umwandlung jedes Dokuments in eine Vektordarstellung kann das LDA-Modell jedes Dokument analysieren und Themen identifizieren, die in jedem Dokument vorkommen. Ohne diese Vektordarstellung könnte das Modell die Dokumente nicht analysieren und Themen identifizieren.

In [4]:
corpus = [dictionary.doc2bow(text) for text in df['preprocessed_text']]

#### 3. Erstellen eines LDA-Modells

In [5]:
from gensim import models

lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)

In [6]:
from src import utils

_ = {
    'model': lda_model,
    'corpus': corpus,
    'dictionary': dictionary
}
utils.safe_as_pkl(_, 'lda_model', '../models')

#### 4. Ergebnisse visualisieren

In [7]:
from src import utils
import pyLDAvis.gensim_models

lda_model = utils.load_pkl('../models/lda_model.pkl')[0]

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model['model'], lda_model['corpus'], lda_model['dictionary'])
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.169041  0.056645       1        1  32.372668
0     -0.164767 -0.247063       2        1  25.148019
3     -0.127099  0.198780       3        1  17.468055
1      0.173962  0.030697       4        1  15.811528
2      0.286944 -0.039059       5        1   9.199730, topic_info=           Term          Freq         Total Category  logprob  loglift
0            ai  65460.000000  65460.000000  Default  30.0000  30.0000
80   blockchain  23249.000000  23249.000000  Default  29.0000  29.0000
339      crypto  12014.000000  12014.000000  Default  28.0000  28.0000
525         nft  10863.000000  10863.000000  Default  27.0000  27.0000
95         web3   8511.000000   8511.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
327     finance    857.946484   1838.528234   Topic5  -5.3192   1.6238
157         day   1039.124811   3725.717364   Topic5  -5.1276   1.1091
0            ai   2054.508801  65460.178406   Topic5  -4.4460  -1.0754
227        nfts    990.598828   5318.056830   Topic5  -5.1754   0.7054
316         get    953.598787   4828.988127   Topic5  -5.2135   0.7638

[370 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
163       1  0.616704    's
163       2  0.147837    's
163       3  0.183507    's
163       4  0.049995    's
163       5  0.001719    's
...     ...       ...   ...
200       1  0.898549  year
200       2  0.055914  year
200       3  0.031451  year
200       4  0.008736  year
200       5  0.004805  year

[750 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 2, 3])

### Evaluation

#### 1. Coherence Score berechnen

Der Coherence Score ist ein Evaluationsmaß für Topic Models, das versucht, die Kohärenz der gefundenen Themen zu bewerten. Die Kohärenz bezieht sich darauf, wie gut die Wörter innerhalb eines Themas zusammenpassen und ob sie eine sinnvolle Bedeutung ergeben. Ein hohes Maß an Kohärenz zeigt an, dass die Themen gut definiert und interpretierbar sind.

In [8]:
from src import utils
from gensim import models

df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')
lda_model = utils.load_pkl('../models/lda_model.pkl')[0]

coherence_model_lda = models.coherencemodel.CoherenceModel(
    model=lda_model['model'], 
    texts=df['preprocessed_text'],
    dictionary=lda_model['dictionary'],
    coherence='c_v')

coherence_score = coherence_model_lda.get_coherence()
coherence_score

0.43644764069027664

#### 2. Hyperparameter Tuning durchführen

Hyperparameter Tuning ist ein wichtiger Schritt im Machine Learning, der dazu beiträgt, das bestmögliche Modell zu finden. In diesem Fall wird das bestmögliche Modell anhand des Coherence Scores bemessen. Ziel des Hyperparameter Tuning ist es demnach, das Modell mit dem höchsten Coherence Score zu finden. Hyperparameter sind Parameter, die das Verhalten des Modells beeinflussen, aber nicht direkt von den Daten gelernt werden. Sie müssen vom Anwender festgelegt werden und können eine erhebliche Auswirkung auf die Leistung des Modells haben. Folgende Parameter sollen optimiert werden:

- num_topics
- alpha
- eta
- passes

**Random Search**

Um nun eine ausreichend gute Kombination dieser Werte zu ermitteln, wird zunächst ein Random-Search Algorithmus ausgeführt. Dieser wählt eine zufällige Kombination an Parametern aus und berechnet für diese den Coherence Score. Ziel des Algorithmus ist es, eine ausreichend gute Menge an Modellen berechnet zu haben, sodass anschließend eine Einschränkung der möglichen Paramenter-Kombinationen vorgenommen werden kann.

In [9]:
%%script false
from src.models.hyperparameter_tuning import RandomSearch

rs = RandomSearch(processed_df='../data/processed/twitter_tweets_processed.feather')
rs.search()

Couldn't find program: 'false'


In [10]:
import pandas as pd

# load results
df = pd.read_feather('../models/hyperparameter_tuning_results_randomsearch.feather')
df.sort_values('coherence_score', ascending=False, inplace=True)
df.head(10)

,random_state,num_topics,alpha,eta,passes,calculation_time,coherence_score
820,1682663347,7,asymmetric,0.7,6,0.8623,0.745585
828,1682663885,22,0.1,0.6,9,1.6366,0.684711
793,1682661172,9,0.1,0.5,9,1.4935,0.629212
569,1682641041,19,0.1,0.6,7,1.2888,0.596950
117,1682601120,12,0.3,0.8,6,1.1865,0.592756
462,1682631941,30,asymmetric,0.2,7,1.4687,0.590857
656,1682648817,24,0.1,0.8,7,1.1790,0.586351
647,1682647932,26,symmetric,0.9,6,1.1454,0.582504
187,1682607279,30,symmetric,0.4,6,1.2631,0.582210
109,1682600378,22,symmetric,0.6,8,1.3561,0.579981


**Grid Search**

Nachdem der Random-Search Algorithmus durchgeführt wurde, lassen sich die ausgewählten Parameter eingrenzen. Nachdem die Eingrenzung vorgenommen wurde, wird mithilfe von Grid-Search für jede Kombination das Modell und der Coherence Score berechnet.

In [11]:
%%script false
from src.models.hyperparameter_tuning import GridSearch

gs = GridSearch(
    processed_df='../data/processed/twitter_tweets_processed.feather', 
    num_topics=[5,6,7], 
    alpha=['asymmetric', 0.1], 
    eta=[0.5, 0.6, 0.7, 0.8, 0.9], 
    passes=[6])
gs.search()

Couldn't find program: 'false'


In [12]:
import pandas as pd

# load results
df = pd.read_feather('../models/hyperparameter_tuning_results_gridsearch.feather')
df.sort_values('coherence_score', ascending=False, inplace=True)
df.head(10)

,random_state,num_topics,alpha,eta,passes,calculation_time,coherence_score
1,1682696636,6,asymmetric,0.6,7,1.0434,0.682241
71,1682702177,14,0.1,0.8,7,1.1283,0.669668
133,1682707459,22,0.1,0.6,7,1.3155,0.664781
11,1682697396,7,asymmetric,0.8,7,0.9854,0.644561
104,1682704947,19,asymmetric,0.5,7,1.2011,0.643404
159,1682709804,25,0.1,0.8,7,1.3418,0.626072
32,1682699045,10,asymmetric,0.5,7,1.1648,0.619837
129,1682707114,22,asymmetric,0.6,7,1.2311,0.607945
70,1682702091,14,0.1,0.7,7,1.2290,0.607379
29,1682698802,9,0.1,0.6,7,1.1825,0.607361


#### 3. Erstellen optimiertes LDA-Modell 

In [13]:
from src import utils
from gensim import models
import pyLDAvis.gensim_models

previous_lda_model = utils.load_pkl('../models/lda_model.pkl')[0]
corpus = previous_lda_model['corpus']
dictionary = previous_lda_model['dictionary']

optimized_lda_model = models.LdaModel(
    corpus=corpus, 
    id2word=dictionary, 
    num_topics=6, 
    alpha='asymmetric', 
    eta=0.6, 
    passes=7, 
    random_state=1682696636)

_ = {
    'model': optimized_lda_model,
    'corpus': corpus,
    'dictionary': dictionary
}

utils.safe_as_pkl(_, 'optimized_lda_model', '../models')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimized_lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.258197  0.137791       1        1  56.023458
3     -0.151347 -0.226800       2        1  17.878587
1      0.086334  0.011324       3        1   8.975768
4      0.056811  0.126159       4        1   8.441228
2      0.122286 -0.004253       5        1   5.832218
5      0.144113 -0.044221       6        1   2.848740, topic_info=            Term          Freq         Total Category  logprob  loglift
80    blockchain  19467.000000  19467.000000  Default  30.0000  30.0000
0             ai  55258.000000  55258.000000  Default  29.0000  29.0000
244     business  11557.000000  11557.000000  Default  28.0000  28.0000
339       crypto  10015.000000  10015.000000  Default  27.0000  27.0000
525          nft   9125.000000   9125.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
649    financial    158.683952    879.931913   Topic6  -5.8345   1.8454
840       update    158.910562    889.716136   Topic6  -5.8331   1.8357
244     business    222.803879  11557.249042   Topic6  -5.4952  -0.3905
2056        gold    106.720141    256.858355   Topic6  -6.2312   2.6800
4245      strong    120.049351    518.767879   Topic6  -6.1135   2.0947

[321 rows x 6 columns], token_table=       Topic      Freq Term
term                       
163        1  0.961779   's
163        2  0.036627   's
163        3  0.000161   's
163        5  0.001285   's
436        1  0.885554  ...
...      ...       ...  ...
873        3  0.000543    “
873        4  0.014663    “
30066      1  0.005163    ✦
30066      3  0.010326    ✦
30066      6  0.981008    ✦

[859 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 3, 6])

---